## Lib

In [1]:
import pandas as pd
import numpy as np

from sktime.classifiers.distance_based import KNeighborsTimeSeriesClassifier
from sktime.classifiers.compose import TimeSeriesForestClassifier
from sktime.classifiers.dictionary_based.boss import BOSSEnsemble

from sktime.classifiers.compose import ColumnEnsembleClassifier

from sktime.utils.load_data import from_long_to_nested

## Data

In [2]:
full_array = np.load('data/Foch_transplant.npy', allow_pickle=True)
col = np.load('data/Foch_transplant_name_col.npy', allow_pickle=True)

In [3]:
df = pd.DataFrame(full_array, columns=col)

# Sktime

In [166]:
df.columns

Index(['Age_donor', 'Aspirations_donor', 'BMI_donor', 'Donneur_CPT',
       'Insuffisance_renale', 'LAS', 'PAPS', 'PFO', 'PF_donor', 'Poids',
       'Poids_donor', 'RX_donor', 'Sex_donor', 'Tabagisme_donor', 'Taille',
       'Taille_donor', 'age', 'body_mass_index', 'diabetes', 'id_patient',
       'other_organ_transplantation', 'oto_score', 'pathologie',
       'plasmapherese', 'preoperative_ECMO', 'preoperative_ICU',
       'preoperative_mechanical_ventilation',
       'preoperative_pulmonary_hypertension', 'preoperative_vasopressor',
       'retransplant', 'sexe', 'super_urgence', 'thoracic_surgery_history',
       'time_on_waiting_liste', 'transplanted_twice_during_study_period',
       'target', 'length_op', 'start_operation_year', 'start_operation_month',
       'start_operation_day', 'ends_operation_year', 'ends_operation_month',
       'ends_operation_day', 'B.I.S', 'BIS SR', 'DC', 'ETCO2', 'FC', 'FR',
       'FiO2', 'PAPdia', 'PAPmoy', 'PAPsys', 'PASd', 'PASm', 'PASs',
       

In [178]:
%%time
dic1 = {'case_id':[], 'dim_id':[], 'reading_id':[], 'value':[]}
nb_max = df['SpO2_by_FiO2'].apply(len).max()
for k, col in enumerate(['B.I.S', 'BIS SR', 'DC', 'ETCO2', 'FC', 'FR',
       'FiO2', 'PAPdia', 'PAPmoy', 'PAPsys', 'PASd', 'PASm', 'PASs',
       'PEEPtotal', 'PNId', 'PNIm', 'PNIs', 'Pmax', 'Pmean', 'SpO2',
       'SvO2 (m)', 'Temp', 'VT', 'SpO2_by_FiO2', 'PASs_minus_PASd',
       'PAPsys_minus_PAPdia', 'declampage_cote1_done',
       'declampage_cote2_done']):
    
    for i, arr in enumerate(df[col]):
        l = len(arr)
        for j, val in enumerate(arr):

            dic1['case_id']+= [i]
            dic1['dim_id']+= [k]
            dic1['reading_id']+= [j]
            dic1['value']+= [val]
        # get ts same length
        for h in range(nb_max-l):
            dic1['case_id']+= [i]
            dic1['dim_id']+= [k]
            dic1['reading_id']+= [l+h]
            dic1['value']+= [0]

Wall time: 6.53 s


In [170]:
ddff1 = pd.DataFrame(dic1)

In [172]:
X_nested1 = from_long_to_nested(ddff1)

X_nested1.columns = ['B.I.S', 'BIS SR', 'DC', 'ETCO2', 'FC', 'FR',
       'FiO2', 'PAPdia', 'PAPmoy', 'PAPsys', 'PASd', 'PASm', 'PASs',
       'PEEPtotal', 'PNId', 'PNIm', 'PNIs', 'Pmax', 'Pmean', 'SpO2',
       'SvO2 (m)', 'Temp', 'VT', 'SpO2_by_FiO2', 'PASs_minus_PASd',
       'PAPsys_minus_PAPdia', 'declampage_cote1_done',
       'declampage_cote2_done']

In [173]:
X_nested1.head()

,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,...,dim_18,dim_19,dim_20,dim_21,dim_22,dim_23,dim_24,dim_25,dim_26,dim_27
0,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 118.0 1 112.0 2 117.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 21.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,...,0 0.0 1 0.0 2 1.0 3 1....,0 99.0 1 99.0 2 97.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 1.000000 1 1.000000 2 4.61...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....
1,0 20.414414 1 20.414414 2 20...,0 0.695961 1 0.695961 2 0.69...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 154.0 1 146.0 2 141.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,...,0 0.0 1 0.0 2 0.0 3 0....,0 99.0 1 100.0 2 100.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 1.0 1 1.0 2 1.0 3 1....,0 67.0 1 59.0 2 62.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....
2,0 20.414414 1 20.414414 2 20...,0 0.695961 1 0.695961 2 0.69...,0 2.229983 1 2.229983 2 2.22...,0 0.0 1 0.0 2 0.0 3 0....,0 111.0 1 122.0 2 112.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 99.0 1 99.0 2 99.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,...,0 1.0 1 1.0 2 1.0 3 1....,0 89.0 1 90.0 2 88.0 3 ...,0 46.775341 1 46.775341 2 46...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.898990 1 0.909091 2 0.88...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....
3,0 20.414414 1 20.414414 2 20...,0 0.695961 1 0.695961 2 0.69...,0 2.229983 1 2.229983 2 2.22...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 97.0 2 94.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 101.0 1 101.0 2 101.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 84.0 2 0.0 3 ...,0 46.775341 1 46.775341 2 46...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.000000 1 0.831683 2 0.00...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....
4,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 110.0 2 111.0 3 ...,0 0.0 1 1.0 2 0.0 3 0....,0 23.0 1 22.0 2 23.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 95.0 2 96.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.000000 1 4.318182 2 4.17...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 22.0 1 22.0 2 22.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....
326,0 20.414414 1 20.414414 2 20...,0 0.695961 1 0.695961 2 0.69...,0 2.229983 1 2.229983 2 2.22...,0 0.0 1 0.0 2 0.0 3 0....,0 99.0 1 93.0 2 100.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 21.0 1 21.0 2 21.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,...,0 0.0 1 0.0 2 0.0 3 0....,0 93.0 1 94.0 2 94.0 3 ...,0 46.775341 1 46.775341 2 46...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 4.428571 1 4.476190 2 4.47...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 

In [161]:
X_nested1.shape

(330, 3)

In [162]:
x_tr = X_nested1[:200][['dim_0']]
y_tr = df['target'][:200].ravel().astype('<U1')

x_te = X_nested1[200:][['dim_0']]
y_te = df['target'][200:].ravel().astype('<U1')

In [177]:
%%time
knn = KNeighborsTimeSeriesClassifier(metric='dtw')
knn.fit(x_tr, y_tr)
knn.score(x_tr, y_tr)

Wall time: 0 ns


0.745

In [ ]:
%time
boss = BOSSEnsemble()
boss.fit(x_tr, y_tr)
boss.score(x_tr, y_tr)

# Multivariate

In [180]:
X_nested1.head()

,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,...,dim_18,dim_19,dim_20,dim_21,dim_22,dim_23,dim_24,dim_25,dim_26,dim_27
0,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 118.0 1 112.0 2 117.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 21.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,...,0 0.0 1 0.0 2 1.0 3 1....,0 99.0 1 99.0 2 97.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 1.000000 1 1.000000 2 4.61...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....
1,0 20.414414 1 20.414414 2 20...,0 0.695961 1 0.695961 2 0.69...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 154.0 1 146.0 2 141.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,...,0 0.0 1 0.0 2 0.0 3 0....,0 99.0 1 100.0 2 100.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 1.0 1 1.0 2 1.0 3 1....,0 67.0 1 59.0 2 62.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....
2,0 20.414414 1 20.414414 2 20...,0 0.695961 1 0.695961 2 0.69...,0 2.229983 1 2.229983 2 2.22...,0 0.0 1 0.0 2 0.0 3 0....,0 111.0 1 122.0 2 112.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 99.0 1 99.0 2 99.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,...,0 1.0 1 1.0 2 1.0 3 1....,0 89.0 1 90.0 2 88.0 3 ...,0 46.775341 1 46.775341 2 46...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.898990 1 0.909091 2 0.88...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....
3,0 20.414414 1 20.414414 2 20...,0 0.695961 1 0.695961 2 0.69...,0 2.229983 1 2.229983 2 2.22...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 97.0 2 94.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 101.0 1 101.0 2 101.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 84.0 2 0.0 3 ...,0 46.775341 1 46.775341 2 46...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.000000 1 0.831683 2 0.00...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....
4,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 110.0 2 111.0 3 ...,0 0.0 1 1.0 2 0.0 3 0....,0 23.0 1 22.0 2 23.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 95.0 2 96.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.000000 1 4.318182 2 4.17...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....


In [191]:
%%time
clf = ColumnEnsembleClassifier(estimators=[
    ("KNN0", KNeighborsTimeSeriesClassifier(metric='dtw'), [0]),
    #("KNN1", KNeighborsTimeSeriesClassifier(metric='dtw'), [1]),
    #("KNN1", KNeighborsTimeSeriesClassifier(metric='dtw'), [2]),
    #("KNN1", KNeighborsTimeSeriesClassifier(metric='dtw'), [3]),
    #("KNN1", KNeighborsTimeSeriesClassifier(metric='dtw'), [4]),
])
clf.fit(X_nested1.head(200), df['target'][:200].ravel().astype('<U1'))
clf.score(X_nested1.tail(100), df['target'][-100:].ravel().astype('<U1'))

Wall time: 18min 30s


0.5

In [190]:
%%time
clf = ColumnEnsembleClassifier(estimators=[
    ("KNN0", KNeighborsTimeSeriesClassifier(metric='dtw'), [0]),
    ("KNN1", KNeighborsTimeSeriesClassifier(metric='dtw'), [1]),
    ("KNN2", KNeighborsTimeSeriesClassifier(metric='dtw'), [2]),
    ("KNN3", KNeighborsTimeSeriesClassifier(metric='dtw'), [3]),
    ("KNN4", KNeighborsTimeSeriesClassifier(metric='dtw'), [4]),
])
clf.fit(X_nested1.head(100), df['target'][:100].ravel().astype('<U1'))
clf.score(X_nested1.tail(100), df['target'][-100:].ravel().astype('<U1'))

Wall time: 33min 51s


0.48